In [1]:
import csv
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
from mysql.connector import MySQLConnection, Error

In [3]:
# Making connection to SQL server
def make_connection():
    try:
        conn = MySQLConnection(host='IES-ADS-ClassDB.sjsu.edu',
                               database='querycrew_db',
                               user='querycrew_user',
                               password='Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [4]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [5]:
cursor.execute("DROP TABLE IF EXISTS driver;")
cursor.execute("DROP TABLE IF EXISTS location;")
cursor.execute("DROP TABLE IF EXISTS vehicle;")
cursor.execute("DROP TABLE IF EXISTS station;")
cursor.execute("DROP TABLE IF EXISTS county;")
cursor.execute("DROP TABLE IF EXISTS IsLocatedIn;") 

conn.commit()
print("All tables dropped successfully!")


All tables dropped successfully!


In [6]:
# Create county table
create_county_table = """
CREATE TABLE county (
    county_id INT NOT NULL PRIMARY KEY,
    county_name VARCHAR(255) NOT NULL
);
"""

# Create station table
create_station_table = """
CREATE TABLE station (
    station_id INT NOT NULL PRIMARY KEY,
    station_name VARCHAR(255) NOT NULL
);
"""

# Create vehicle table
create_vehicle_table = """
CREATE TABLE vehicle (
    vin VARCHAR(17) NOT NULL PRIMARY KEY,
    model_year INT NOT NULL,
    base_msrp DECIMAL(10, 2),
    make VARCHAR(100) NOT NULL,
    model VARCHAR(100) NOT NULL
);
"""

# Create location table
create_location_table = """
CREATE TABLE location (
    DOL_ID INT NOT NULL PRIMARY KEY,
    zip_code VARCHAR(10) NOT NULL,
    city VARCHAR(100) NOT NULL,
    state VARCHAR(50) NOT NULL,
    vin VARCHAR(17) NOT NULL,
    station_id INT NOT NULL,
    county_id INT NOT NULL,
    FOREIGN KEY (vin) REFERENCES vehicle(vin),
    FOREIGN KEY (station_id) REFERENCES station(station_id),
    FOREIGN KEY (county_id) REFERENCES county(county_id)
);
"""

# Create driver table
create_driver_table = """
CREATE TABLE driver (
    email VARCHAR(255) NOT NULL PRIMARY KEY,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    vin VARCHAR(17) NOT NULL,
    FOREIGN KEY (vin) REFERENCES vehicle(vin)
);
"""

# Execute table creation
cursor.execute(create_county_table)
cursor.execute(create_station_table)
cursor.execute(create_vehicle_table)
cursor.execute(create_location_table)
cursor.execute(create_driver_table)

conn.commit()
print("Tables county, station, vehicle, location, and driver created successfully!")


Tables county, station, vehicle, location, and driver created successfully!


In [7]:
# Insert data into county
insert_county = """
INSERT INTO county (county_id, county_name) VALUES 
(1, 'Santa Clara'), (2, 'Alameda'), (3, 'San Francisco'), (4, 'San Mateo'), (5, 'Contra Costa');
"""

# Insert data into station
insert_station = """
INSERT INTO station (station_id, station_name) VALUES 
(101, 'San Jose Test Center'), 
(102, 'Oakland Inspection Hub'), 
(103, 'San Francisco Checkpoint'), 
(104, 'Redwood City Station'), 
(105, 'Fremont Auto Test');
"""

# Insert data into vehicle
insert_vehicle = """
INSERT INTO vehicle (vin, model_year, base_msrp, make, model) VALUES 
('1HGBH41JXMN109186', 2020, 35000.00, 'Toyota', 'Camry'),
('2FTHF26H9LC123456', 2019, 42000.00, 'Ford', 'F-150'),
('3N1AB7AP3JL567890', 2021, 25000.00, 'Nissan', 'Sentra'),
('4T1BF1FK0HU765432', 2022, 28000.00, 'Honda', 'Accord'),
('5YJSA1E26HF789012', 2023, 55000.00, 'Tesla', 'Model S');
"""

cursor.execute(insert_county)
cursor.execute(insert_station)
cursor.execute(insert_vehicle)
conn.commit()

print("Data inserted into county, station, and vehicle successfully!")


Data inserted into county, station, and vehicle successfully!


In [8]:
insert_location = """
INSERT INTO location (DOL_ID, zip_code, city, state, vin, station_id, county_id) VALUES 
(301, '95112', 'San Jose', 'CA', '1HGBH41JXMN109186', 101, 1),
(302, '94601', 'Oakland', 'CA', '2FTHF26H9LC123456', 102, 2),
(303, '94103', 'San Francisco', 'CA', '3N1AB7AP3JL567890', 103, 3),
(304, '94063', 'Redwood City', 'CA', '4T1BF1FK0HU765432', 104, 4),
(305, '94536', 'Fremont', 'CA', '5YJSA1E26HF789012', 105, 5);
"""

cursor.execute(insert_location)
conn.commit()
print("Data inserted into location successfully!")


Data inserted into location successfully!


In [9]:
insert_driver = """
INSERT INTO driver (email, first_name, last_name, vin) VALUES 
('john.doe@gmail.com', 'John', 'Doe', '1HGBH41JXMN109186'),
('jane.smith@gmail.com', 'Jane', 'Smith', '2FTHF26H9LC123456'),
('alice.johnson@gmail.com', 'Alice', 'Johnson', '3N1AB7AP3JL567890'),
('bob.brown@gmail.com', 'Bob', 'Brown', '4T1BF1FK0HU765432'),
('charlie.davis@gmail.com', 'Charlie', 'Davis', '5YJSA1E26HF789012');
"""

cursor.execute(insert_driver)
conn.commit()
print("Data inserted into driver successfully!")


Data inserted into driver successfully!


In [10]:
import pandas as pd

# Function to display table contents
def display_table(table_name):
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql(query, conn)
    print(f"\nTable: {table_name}")
    display(df)

# Display location and driver tables
display_table("location")
display_table("driver")

# Close connection
conn.close()
print("Connection closed.")



Table: location


/Users/keon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DOL_ID,zip_code,city,state,vin,station_id,county_id
0,301,95112,San Jose,CA,1HGBH41JXMN109186,101,1
1,302,94601,Oakland,CA,2FTHF26H9LC123456,102,2
2,303,94103,San Francisco,CA,3N1AB7AP3JL567890,103,3
3,304,94063,Redwood City,CA,4T1BF1FK0HU765432,104,4
4,305,94536,Fremont,CA,5YJSA1E26HF789012,105,5



Table: driver


/Users/keon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,email,first_name,last_name,vin
0,alice.johnson@gmail.com,Alice,Johnson,3N1AB7AP3JL567890
1,bob.brown@gmail.com,Bob,Brown,4T1BF1FK0HU765432
2,charlie.davis@gmail.com,Charlie,Davis,5YJSA1E26HF789012
3,jane.smith@gmail.com,Jane,Smith,2FTHF26H9LC123456
4,john.doe@gmail.com,John,Doe,1HGBH41JXMN109186


Connection closed.
